## Import and Cleanup Data

In [1]:
import pandas as pd
import pickle as pkl
import numpy as np
from sklearn.decomposition import PCA, FactorAnalysis
from factor_analyzer import FactorAnalyzer
from sklearn.cross_decomposition import CCA
import matplotlib.pyplot as plt
from mca import MCA


In [4]:
## If data already preprocessed, load it in, else do preprocessing

try:
    data = pd.read_csv('./data/Lindel_alldata.csv', index_col=0)
    print("Data successfully loaded")
except Exception:
    label, rev_index, features = pkl.load(open('./data/feature_index_all.pkl','rb'))
    Lindel_training = pd.read_csv("./data/Lindel_training_65bp.csv", sep=',', index_col=0)
    Lindel_test = pd.read_csv("./data/Lindel_test_65bp.csv", sep=',', index_col=0)

    print("Number of labels : ", len(label.keys()))
    print("Number of rev_index : ", len(rev_index.keys()))
    print("Number of features : ", len(features.keys()))

    # column descriptions
    # Lindel_training.iloc[0] # guide sequences
    # Lindel_training.iloc[1:3034] # 3033 binary features [2649 MH binary features + 384 one hot encoded features]
    # Lindel_training.iloc[3034:] # 557 observed outcome frequencies

    # # Merge training and test set for dimensionality reduction
    all_data = pd.concat([Lindel_training, Lindel_test])
    # data_features = all_data.iloc[:, 1:3034]

    # # Clean up data
    features = dict(sorted(features.items(), key=lambda item: item[1]))
    feature_labels = list(features.keys())

    labels = dict(sorted(label.items(), key=lambda item: item[1]))
    class_labels = list(labels.keys())

    one_hot_labels = []
    for i in range(80):
        one_hot_labels.append("nt {}".format(str(int(i / 4) + 1)))

    for i in range(304):
        one_hot_labels.append("2nt {}".format(str(int(i / 16) + 1)))

    one_hot_labels = np.array(one_hot_labels)

    column_labels = np.concatenate((np.array(['Guide Sequence', '65bp']), feature_labels, one_hot_labels, class_labels))

    # Rename columns of test and training set
    Lindel_training = Lindel_training.set_axis(column_labels, axis=1, inplace=False)
    Lindel_test = Lindel_test.set_axis(column_labels, axis=1, inplace=False)

    data = pd.concat([Lindel_training, Lindel_test], axis=0)


Data successfully loaded


In [5]:
data

,Guide Sequence,65bp,-30+29+0,-30+29+1,-30+29+2,-30+29+3,-30+29+4,-29+29+0,-29+29+1,-29+29+2,...,2+TG,2+CA,2+CT,2+CC,2+CG,2+GA,2+GT,2+GC,2+GG,3
0,CGGTCTCTATCAGTCCGGAA,TAACGTTATCAACCGGTCTCTATCAGTCCGGAACGGTTGAACTGCG...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.01833,0.0,0.0,0.0,0.0,0.057026
1,GCTTGCTGGCCTCGGACTAT,TAACGTTATCAACGCTTGCTGGCCTCGGACTATCGGTTGAACTGCG...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000
2,GTTCAGGGCACTCGCGAATT,TAACGTTATCAACGTTCAGGGCACTCGCGAATTCGGTTGAACTGCG...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.040000
3,GCTGATCAAGCGCGGTTCAG,TAACGTTATCAACGCTGATCAAGCGCGGTTCAGCGGTTGAACTGCG...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.067797
4,GCGTCCGTGAGAATCCTATA,TAACGTTATCAACGCGTCCGTGAGAATCCTATACGGTTGAACTGCG...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,CTGTGAGTTAACTTCGGCAA,TAACGTTATCAACCTGTGAGTTAACTTCGGCAACGGTTGAACTGCG...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.142857
435,CGACCTATAGCGGCCCGGAC,TAACGTTATCAACCGACCTATAGCGGCCCGGACCGGTTGAACTGCG...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.000000,0.055556,0.00000,0.0,0.0,0.0,0.0,0.000000
436,CCTAAGCGCATACACGGTCC,TAACGTTATCAACCCTAAGCGCATACACGGTCCCGGTTGAACTGCG...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000
437,CGCGAGTCGGTAGACGGCAC,TAACGTTATCAACCGCGAGTCGGTAGACGGCACCGGTTGAACTGCG...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.034014


# RUN DIMENSIONALITY REDUCTIONS

In [ ]:
# 1 : PCA

# Instantiate PCA
n_components = 910
pca = PCA(n_components=n_components)

# Determine transformed features
X_train_pca = pca.fit_transform(data.iloc[:, 1:])

# Determine explained variance using explained_variance_ration_ attribute
exp_var_pca = pca.explained_variance_ratio_

# Cumulative sum of eigenvalues; This will be used to create step plot
# for visualizing the variance explained by each principal component.
cum_sum_eigenvalues = np.cumsum(exp_var_pca)
print("Nr of components : ", n_components)
print("Total Percentage of variance explained = ", cum_sum_eigenvalues[-1] * 100)
#
# Create the visualization plot
#b
plt.bar(range(0,len(exp_var_pca)), exp_var_pca, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid',label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
# 2 : MCA
"""
Multiple correspondence analysis (MCA) is an extension of correspondence analysis (CA).
It should be used when you have more than two categorical variables.
The idea is simply to compute the one-hot encoded version of a dataset and apply CA on it.
As an example we're going to use the balloons dataset taken from the UCI datasets website.
"""
mca_obj = MCA(data.iloc[:, 1:])

row_factor_scores = mca_obj.fs_r(percent=0.99)
print("Row factor scores (99% variance explained) shape : ", row_factor_scores.shape)

column_factor_scores = mca_obj.fs_c(percent=0.99)
print("Column factor scores (99% variance explained) shape : ", column_factor_scores.shape)


In [ ]:
# 3 : FA

# Instantiate factor analysis object
from factor_analyzer.factor_analyzer import FactorAnalyzer 
fa = FactorAnalyzer(rotation='varimax')
fa.fit(data.iloc[:, 1:])
# Check Eigenvalues
ev, v = fa.get_eigenvalues()
ev